In [1]:
%matplotlib inline

import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

from auxiliary import *

np.random.seed(123)

# Self-selection, heterogeneity, and causal graphs

**Overview**

* Introduction

* Nonignorability and selection on the unobservables revisited

* Selection on the unobservables and the utility of additional posttreatment measures of the outcome

* Causal graphs for complex patterns of self-selection and heterogeneity

* Conclusion

**Alternatives to back-door identification**

The next chapters deal with:

* instrumental variables
* front-door identification with causal mechanisms
* conditioning estimators using pretreatment variables

Why do we need to consider alternatives?

$\rightarrow$selection on unobservables / nonignorability of treatment

What makes an unobservable?

* simple confounding, stable unobserved common cause of treatment and outcome variable

* subtle confounding, direct self-selection into the treatment based on accurate perceptions of 
the individual level treatment effect

Selection on unobservables as a combination of two features:

* treatment effect heterogeneity
* self-selection

## Nonignorability and selection on the unobservables

### Selection on observables 

<img src="material/figure-4-8.png" height=500 width=500 />

### Selection on unobservables

<img src="material/figure-4-9.png" height=500 width=500 />

## Selection on the unobservables and the utility of additional posttreatment measures of the outcome

**Catholic school example**

* claim that Catholic schools are more effective than public schools in teaching mathematics and 
reading to equivalent High School students.

* conditioning on family background and motivation to learn 

* those enrolling into Catholic school have the most to gain from doing so

**Notation**

* $Y_{10}$, observed score on standardized achievement test given in tenth grade

* $D$ causal variable taking value one if student attends Catholic school

* $U$ unobserved motivation to learn, differences in home environment, anticipation of causal 
effect itself


We proceed in two steps:

* assess identification for given directed graphs

* examine structure of directed graph itself



<img src="material/figure-8-1.png" height=500 width=500 />

We cannot identify the causal effect of $D$ on $Y_{10}$ in subfigure (a) but in subfigure (b).
However, at what cost?

$Y_{10}$ does not satisfy the Condition 2 of the back-door criterion. As such, it adjusts away 
some of the total causal effect of $D$ on $Y_{12}$.

Let $E$ denote an student's ability for test taking. Then maybe this is a more complete picture?

<img src="material/figure-8-2.png" height=500 width=500 />

Back-door adjustment by $Y_{10}$ ineffective again after revisiting economic implications of the 
imposed graph. In fact, $Y_{10}$ is now a collider variable that induces a noncausal dependence.

## Panel Data Demonstration

The motivation behind this example is simply to show that we cannot learn anything about the 
underlying causal effect with the conventional strategies and how we model self-selection in the 
data generating process.

In [2]:
def get_propensity_score(o, u):
    """Get the propensity score."""
    level = -3.8 + o + u 
    return np.exp(level) / (1 + np.exp(level))


def get_treatment_status(o, u):
    """Sampling treatment status"""
    # Following the causal graph, the treatment indicator is only a function 
    # of the background characteristics O and the unobservable U.
    p = get_propensity_score(o, u)
    return np.random.choice([1, 0], p=[p, 1 - p])
   
def get_covariates():
    """ Get covariates."""
    o, e = np.random.normal(size=2)
    x, u = o + np.random.normal(size=2)    
    return o, u, x, e

In [3]:
def get_potential_outcomes(grade, o, x, e, u, scenario=0, selection=False):
    """Get potential outcomes.
    
    Sampling of potential outcome of an individual for the panel data demonstration.
    
    Args:
        grade: an integer for the grade the individual is in.
        o, x, e : floats of observable characteristics.
        u: a float of unobservable characteristic.
        scenario: an integer for the scenario: (0) no role for E, (1) role for E.
        selection: a boolean indicating whether there is selection on unobservables.
    
    Returns:
        A tuple of potential outcomes (Y_0, Y_1).
    
    """
    # We want to make sure we only pass in valid input.
    assert scenario in range(2)
    assert selection in [True, False]
    assert grade in [10, 11, 12]
    
    # There is a natural progression in test scores.
    level = dict()
    level[10] = 100
    level[11] = 101
    level[12] = 102
        
    if scenario == 0:
        y_0 = level[grade] + o + u + x + np.random.normal()
    elif scenario == 1:
        y_0 = level[grade] + o + u + x + e + np.random.normal()
    else:
        raise NotImplementedError
    
    # Sampling of treatment effects. The key difference for selection on unobservables is in how 
    # the overall treatment effect depends on the  unobservable U that also affects the choice 
    # probability. This was the major criticism of Coleman's work.
    delta_1 = np.random.normal(loc=10, scale=1)
    if selection:
        delta_2 = np.random.normal(loc=u)
    else:
        delta_2 = np.random.normal()
    
    if grade == 10:
        y_1 = y_0 + delta_1 + delta_2
    elif grade == 11:
        y_1 = y_0 + (1 + delta_1) + delta_2
    elif grade == 12:
        y_1 = y_0 + (2 + delta_1) + delta_2
        
    return y_0, y_1

In [4]:
def get_sample_panel_demonstration(num_agents=1000, scenario=0, selection=False, seed=123):
    """Get sample for demonstration.
    
    Create a random sample for the demonstration of the usefulness of (or lack thereof) of having
    additional posttreatment measures of the outcome.
    
    Args:
        num_agents: an integer for the number of agents in the sample
        scenario: an integer that indicates whether to include E as a determinant of test scores
        selection: a boolean variable indicating whether selection on unobservables is an issue
        seed: an integer setting the random seed
        
    Returns:
        A dataframe with the simulated sample.
    
    """
    # We first initialize an empty DataFrame that holds the information for each individual
    # and each time period.
    columns = ['Y', 'D', 'O','X', 'E', 'U', 'Y_1', 'Y_0']
    index = list()
    for i in range(num_agents):
        for j in [10, 11, 12]:
            index.append((i, j))
    index = pd.MultiIndex.from_tuples(index, names=('Identifier', 'Grade'))
    df = pd.DataFrame(columns=columns, index=index)

    # Now we are ready to simulate the sample with the desired characteristics.
    np.random.seed(seed)
    for i in range(num_agents):

        o, u, x, e = get_covariates()
        d = get_treatment_status(o, u)
        for grade in [10, 11, 12]:
            y_0, y_1 = get_potential_outcomes(grade, o, u, x, e, scenario, selection)
            y = d * y_1 + (1 - d) * y_0
            df.loc[(i, grade), :] = [y, d, o, x, e, u, y_1, y_0]
    
    # Finally some type definitions for pretty output.
    df = df.astype(np.float)
    df = df.astype({'D': np.int})

    return df

In [5]:
num_agents, scenario, selection = 1000, 0, False
df = get_sample_panel_demonstration(num_agents, scenario, selection)
df.head()

Y  D         O         X         E         U  \
Identifier Grade                                                          
0          10      97.641896  0 -1.085631 -0.802652  0.997345 -2.591925   
           11     100.299138  0 -1.085631 -0.802652  0.997345 -2.591925   
           12      98.872349  0 -1.085631 -0.802652  0.997345 -2.591925   
1          10      96.817222  0 -0.619191 -0.042445 -0.769433 -0.492665   
           11      99.856079  0 -0.619191 -0.042445 -0.769433 -0.492665   

                         Y_1         Y_0  
Identifier Grade                          
0          10     107.386177   97.641896  
           11     108.565873  100.299138  
           12     112.397378   98.872349  
1          10     108.207462   96.817222  
           11     111.410977   99.856079

What is the average treatment effect and how does it depend on the presence of selection?

In [6]:
num_agents, scenario = 1000, 0

# This setup allows to freeze some arguments of the function
# that do not change during the analysis.
from functools import partial
simulate_sample = partial(get_sample_panel_demonstration, num_agents, scenario)

for selection in [False, True]:
    print(' Selection {:}'.format(selection))
    df = simulate_sample(selection)
    for grade in [10, 12]:
        subset = df.loc[(slice(None), grade), :]
        stat = (subset['Y_1'] - subset['Y_0']).mean()
        print(" Grade {:}:  ATE {:5.3f}".format(*[grade, stat]))
    print('\n')

 Selection False
 Grade 10:  ATE 10.090
 Grade 12:  ATE 12.014


 Selection True
 Grade 10:  ATE 10.100
 Grade 12:  ATE 12.023




The average treatment effects are unaffected by selection.

In [7]:
for selection in [False, True]:
    print(' Selection {:}'.format(selection))
    df = simulate_sample(selection)
    for grade in [10, 12]:
        subset = df.loc[(slice(None), grade), :]
            
        treated = subset['D'] == 1
        control = subset['D'] == 0

        stat = list()
        stat.append((subset['Y_1'][treated] - subset['Y_0'][treated]).mean())
        stat.append((subset['Y_1'][control] - subset['Y_0'][control]).mean())
        print(" Grade {:}:  ATT {:5.3f}   ATC {:5.3f}".format(grade, *stat))
    print('\n')

 Selection False
 Grade 10:  ATT 10.012   ATC 10.098
 Grade 12:  ATT 12.134   ATC 12.001


 Selection True
 Grade 10:  ATT 9.894   ATC 10.121
 Grade 12:  ATT 12.016   ATC 12.024




In [8]:
for grade in [10, 12]:
    for model in ['Y ~ D', 'Y ~ D + X + O']:
        subset = df.loc[(slice(None), grade), :]
        rslt = smf.ols(formula=model, data=subset).fit()
        stat = rslt.params['D']
        print('Grade: {}  Model: {:}'.format(*[grade, model]))
        print('   Estimated Treatment Effect: {:5.3f}\n'.format(stat))

Grade: 10  Model: Y ~ D
   Estimated Treatment Effect: 12.650

Grade: 10  Model: Y ~ D + X + O
   Estimated Treatment Effect: 10.315

Grade: 12  Model: Y ~ D
   Estimated Treatment Effect: 14.731

Grade: 12  Model: Y ~ D + X + O
   Estimated Treatment Effect: 12.466



None of the estimates come even close to our parameters of interest.

## Causal graphs for complex patterns of self-selection

We want to make sure that complex patterns of self-selection can be represented by directed graphs.


### Separate graphs for separate latent classes

**Groups**

* $G=1$, selection of schools mainly for lifestyle reasons, proximity to home and taste for 
school cultures

* $G=2$, selection of schools to maximize expected achievement

<img src="material/figure-8-3.png" height=500 width=500 />


What are the economic mechanisms are represented by each of the arrows? Why would we expect them 
to differ across the two groups?

* families of the second group are more likely to send their children to charter schools $d_2 > d_1$

* parents with higher levels of education are more likely to send their children to charter  schools
 as they value distinctive forms of education $\alpha_1, \alpha_2 > 0$ and are able to support 
 their children with their homework $\beta_1, \beta_2 > 0$.

* existing research suggests $\delta_1, \delta_2 > 0$ and $\delta_2 > \delta_1$ as families in 
second group put more effort in matching their children to schools

What happens if we block the back-door path by conditioning in $P$ but ignore the existence of 
two latent classes? If $P$ is associated with latent class membership, then we do not properly 
weigh the stratum-specific treatment effects as there is heterogeneity within strata.

### A single graph that represents all latent classes

<img src="material/figure-8-4.png" height=500 width=500 />

<img src="material/figure-8-5.png" height=500 width=500 />

### Self-selection into the latent class

<img src="material/figure-8-6.png" height=500 width=500 />

In [ ]:
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen('http://bit.ly/2DeuEQx')
HTML(html.read().decode('utf-8'))